# Translating CASRNs to InChI and/or PubChem CID

Although nobody in the open data world really cares about CASRNs, they are an industry standard, and in particular environmental chemical information is very often indexed to them. Example: [US EPA ACToR database](http://actor.epa.gov/actor/faces/ACToRHome.jsp).

This notebook will distill what I've learned from trying to use the following resources to do CASRN → structure or open identifier translation:

* [PubChem PUG REST API](https://pubchem.ncbi.nlm.nih.gov/pug_rest/PUG_REST.html). There is a way to look up compounds by CASRN, in the compound domain namespace `xref/RN`.
* [PubChem Identifier Exchange Web Service](https://pubchem.ncbi.nlm.nih.gov/idexchange) (which I am calling "IXS" for short). Tis is a web form, not an API.
* [NLM Chemical Identifier Resolver](http://cactus.nci.nih.gov/chemical/structure) (CIR). This can give you StdInChI/Key directly, without looking up CID (in fact it doesn't give you CID at all).

These resources only have approximate correspondences between CASRN and other IDs. As a result, conversions are sloppy and there are lots of misalignments that need to be corrected by further inspection of the results.

The CIR is not very reliable at CASRN → InChI conversions in my experience. They also probably ultimately get their data from PubChem, both being National Library of Medicine projects.

I have not made a systematic comparison between the PubChem PUG and IXS services, but I have generated a dataset that can be used to make such a comparison (below). Using PUG for repeated lookups of many CASRNs is slow and inefficient.


## My conclusions, at least for now
* The PubChem IXS is the easiest way to make a sloppy conversion of many identifiers at once. It is faster and easier than using the web APIs (which are more prone to connection errors).
* The work of aligning CASRNs with other IDs is impossible without some human input, and also requires knowing some more details about what the chemicals in question are (i.e., names and/or structures).
    - To this end, I've shifted my efforts to developing a tool to help us do that quickly.

Imports and stuff:

In [1]:
from __future__ import unicode_literals, print_function
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import requests
from chemex.casrn import validate

## Using PubChem services to convert a list of CASRNs to CIDs

I wanted to see how the PUG REST API lookup in the `xref/RN` namespace compares to the results of the automated IXS, which interprets CASRN as _synonyms_. 

Note: Synonyms, including CASRNs, are deposited in an uncontrolled and uncurated manner for each compound in PubChem. They do not constitute a namespace for the PubChem API, but PubChem maintains a cross-reference of CIDs and all their synonyms ([huge file, FTP](ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/CID-Synonym-filtered.gz)). Instead of diving right into that huge file, I tried a smaller experiment.

So... Here's what I did. 
* I took the CASRN column of the [Calfiornia Proposition 65 List](http://oehha.ca.gov/prop65/prop65_list/newlist.html) (an outdated version now, it doesn't matter) and cleaned it, dropping duplicates and fixing bad CASRN strings, etc.
    - pickled DataFrame: `../data/p65.pickle`
    - CASRN only: `../data/p65_CASRN.csv`.
* Used this plain list of CASRN to generate a corresponding list of PubChem CIDs using
    - the PUG REST API → `../data/p65_CASRN_CID_api.csv`
    - the IXS → `../data/p65_CASRN_CID_ixs.txt`.

For each step that generates significant output, the code below includes statements to write files in `../results/`. When I ran this I saved a copy of each file in `../data/` so you can look there if you want to see. 

*Also: Unicode errors with pandas to_csv? Try it in Python 3...*

In [2]:
xls_file = pd.ExcelFile('../data/P65list040414.xls')
df = xls_file.parse('Sheet1', skiprows=13, skip_footer=6, na_values=['--', '---', ' ---', '  ---'], 
                    names=['ChemName', 'Toxicity', 'ListMech', 'CASRN', 'DateListed', 'NSRL_MADL'])
p65 = df.loc[:,['ChemName', 'CASRN']]
p65 = p65[p65.CASRN.notnull()].sort(columns='CASRN')
p65['CASRN'] = p65['CASRN'].apply(validate)
p65.dropna(inplace=True)
p65.drop_duplicates(subset='CASRN', inplace=True, take_last=True)
print(len(p65))
p65.head()

715


,ChemName,CASRN
176,1-Chloro-4-nitrobenzene,100-00-5
344,p-Dinitrobenzene,100-25-4
921,4-Vinylcyclohexene,100-40-3
397,Ethylbenzene,100-41-4
89,Benzyl chloride,100-44-7


In [ ]:
# Previous results are saved in ../data/p65.pickle
p65.to_pickle('../results/p65.pickle')

In [ ]:
# Previous results are saved in ../data/p65_CASRN.csv
p65['CASRN'].to_csv('../results/p65_CASRN.csv', index=False)

### PubChem Identifier Exchange Web Service ("IXS")

I used IXS to convert that list (715 lines) to CIDs and saved the output to `../data/p65_CASRN_CID_ixs.txt` (979 lines) 

I used the following settings:

* Input ID type: Synonym 
* Operator type: Same CID
* Output type: CID
* Output method: Two column file showing each input-output correspondence
* Compression: no compression

In [3]:
p65_cids_ixs = pd.read_table('../data/p65_CASRN_CID_ixs.txt', index_col=None, names=['CASRN', 'CID_ixs'])
print(len(p65_cids_ixs))
p65_cids_ixs.head()

979


,CASRN,CID_ixs
0,100-00-5,7474
1,100-25-4,7492
2,100-40-3,7499
3,100-41-4,7500
4,100-44-7,7503


There are one-to-many relationships between CASRNs and CIDs. For example, "10418-03-8" is a synonym for four PubChem compounds:

In [4]:
p65_cids_ixs[p65_cids_ixs.CASRN == '10418-03-8']

,CASRN,CID_ixs
21,10418-03-8,25249
22,10418-03-8,5277
23,10418-03-8,44629810
24,10418-03-8,12866474


See below (*Comparing results*) for more on the degree of one-to-many-ness...

### PUG REST API
It's possible to generate a comparable dataset by using the PubChem API directly. However, it takes *much* more time and effort. The following code shows how I did this.

In [5]:
import os
from time import sleep

pc_default_props = 'IUPACName,MolecularFormula,InChIKey'

def pc_get_cid_properties(data, properties=pc_default_props, verbose=False):

    while data['CID']:
        cid = data['CID']
        try:
            r = requests.get('http://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{0}/property/{1}/JSON'
                             .format(cid, properties)).json()
        except ConnectionError as e:
            if verbose:
                print('Connection error while requesting properties for CID {0}: {1}'.format(cid, e))
                print('Trying again...')
            sleep(15)
            continue
        else:
            data.update(r['PropertyTable']['Properties'][0])
            break

    return data


def pc_casrn_cid_lookup(casrn, properties=pc_default_props, verbose=False):
    
    casrn = validate(casrn)
    if verbose:
        print(casrn)
    
    results = []

    # CASRN-to-CIDs lookup using PubChem API.
    while True:
        try:
            r = requests.get('http://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/xref/RN/{0}/JSON'
                             .format(casrn)).json()
        except ConnectionError as e:
            if verbose:
                print('Connection error while requesting CIDs for CASRN {0}: {1}'.format(casrn, e))
                print('Trying again...')
            sleep(15)
            continue
        else:
            break

    # Wait before making the next API call -- not sure if this is necessary.
    sleep(0.1)
    
    # Add all CIDs (if any) to the results & look up additional properties.
    try:
        cpds = r['PC_Compounds']
    except KeyError as e:
        if verbose:
            print('No data for CASRN {0}: {1}'.format(casrn, e))
        results.append({'CASRN': casrn})
    else:
        for c in cpds:
            data = {'CASRN': casrn, 'CID': c['id']['id']['cid']}
            # Retrieve the desired properties, if any.
            if properties:
                data = pc_get_cid_properties(data, verbose=verbose)
            results.append(data)
    
    return results


## I think this could be done in a more sophisitcated way with a class.

def pc_casrn_chunks(df, name, startk=0, endk=False, k=50, col='CASRN',
                    properties=pc_default_props, verbose=False, d=30):

    totl = len(df[col])
    nchunks = int(np.ceil(totl/k))
    stopk = endk or nchunks
    if verbose:
        print('Name:', name)
        print('Total: {0} CASRN, {1} chunks. Retrieving chunks {2} to {3}'
              .format(totl, nchunks, startk, stopk - 1))

    chunks_dict = {n: '../temp/{0}_{1}.pkl'.format(name, n) for n in range(startk, stopk)}
    
    for n in range(startk, stopk):
        # Skips chunks that have already been retrieved.
        if os.path.exists(chunks_dict[n]):
            if verbose:
                print('Already have chunk {0}: {1}'.format(n, chunks_dict[n]))
            continue

        results = []
        # Establish the start & end elements of this chunk.
        i, j = n * k, np.minimum(((n + 1) * k), totl)
        if verbose:
            print('Getting chunk {0}, rows {1} to {2}'.format(n, i, j))
        # Request the data
        for c in df[col][i:j]:
            data = pc_casrn_cid_lookup(c, properties=properties, verbose=verbose)
            for x in data:
                results.append(x)
        
        # Store data before next call
        new_df = DataFrame(results)
        
        # After every request pickle the dataframe
        new_df.to_pickle(chunks_dict[n])
        
        # Print status
        if verbose:
            print('Retrieved chunk {0}: {1}'.format(n, chunks_dict[n]))
        
        # Wait before the next loop (default 30 s)
        if n < stopk - 1:
            sleep(d)

    if verbose:
        print('Done.')

    return chunks_dict


def combine_chunks(chunks_dict):
    
    frames = []
    
    for i in chunks_dict.keys():
        df = pd.read_pickle(chunks_dict[i])
        frames.append(df)

    return pd.concat(frames).reset_index().drop('index', axis=1)

Here are some short tests:

In [6]:
pc_casrn_cid_lookup('100-44-7', verbose=True)

100-44-7


[{'CASRN': '100-44-7',
  'CID': 7503,
  'IUPACName': 'chloromethylbenzene',
  'InChIKey': 'KCXMKQUNVWSEMD-UHFFFAOYSA-N',
  'MolecularFormula': 'C7H7Cl'},
 {'CASRN': '100-44-7',
  'CID': 272643,
  'IUPACName': '2,2-bis(4-methylphenyl)cyclopropane-1-carboxylic acid',
  'InChIKey': 'MPFKAXRDUNPKDV-UHFFFAOYSA-N',
  'MolecularFormula': 'C18H18O2'}]

In [7]:
DataFrame(pc_casrn_cid_lookup('10418-03-8', verbose=True))

10418-03-8


,CASRN,CID,IUPACName,InChIKey,MolecularFormula
0,10418-03-8,5277,NaN,LKAJKIOFIWVMDJ-UHFFFAOYSA-N,C21H32N2O
1,10418-03-8,25249,NaN,LKAJKIOFIWVMDJ-IYRCEVNGSA-N,C21H32N2O
2,10418-03-8,237730,propan-2-yl 4-(3-chloropropanoylamino)benzoate,FYSWBTKTCGJURE-UHFFFAOYSA-N,C13H16ClNO3
3,10418-03-8,12866474,NaN,LKAJKIOFIWVMDJ-GVQHXQKQSA-N,C21H32N2O
4,10418-03-8,44629810,NaN,LKAJKIOFIWVMDJ-OLPJAUBDSA-N,C21H32N2O


I needed to write the chunky function `pc_casrn_chunks` because making many API calls inevitably resulted in connection errors, and try as I may, I don't know a better way to deal with those. This is probably not the best implementation.

**Running the following code will take a while!** And I already ran it: and the results are in `../data/p65_CASRN_CID_api.csv`.

In [ ]:
p65_pickles = pc_casrn_chunks(p65, 'p65_pickle', verbose=True)

In [ ]:
p65_cids_api = combine_chunks(p65_pickles)

In [ ]:
# Previous results are saved in `../data/p65_CASRN_CID_api.csv`.
p65_cids_api.to_csv('../results/p65_CASRN_CID_api.csv', index=False)

### Results from PUG REST API lookups

In [8]:
p65_cids_api = pd.read_csv('../data/p65_CASRN_CID_api.csv')
print(len(p65_cids_api))
p65_cids_api.head()

1895


,CASRN,CID,IUPACName,InChIKey,MolecularFormula
0,100-00-5,7474,1-chloro-4-nitrobenzene,CZGCEKJOLUNIFY-UHFFFAOYSA-N,C6H4ClNO2
1,100-00-5,101111,"(2S)-2-[(3,5-dinitrobenzoyl)amino]-4-methylpen...",DIOBIOPCRMWGAT-NSHDSACASA-N,C13H15N3O7
2,100-25-4,7492,"1,4-dinitrobenzene",FYFDQJRXFWGIBS-UHFFFAOYSA-N,C6H4N2O4
3,100-40-3,7499,4-ethenylcyclohexene,BBDKZWKEPDTENS-UHFFFAOYSA-N,C8H12
4,100-41-4,7500,ethylbenzene,YNQLUTRBYVCPMQ-UHFFFAOYSA-N,C8H10


### Comparing results

There are nearly twice as many results from using the PUG API as the IXS.

We can also examine the degree to which there are multiple CIDs for each CASRN:

In [9]:
p65_cids_ixs.groupby('CASRN').apply(len).describe()

count    694.000000
mean       1.410663
std        1.301548
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max       13.000000
dtype: float64

In [10]:
p65_cids_api[['CASRN', 'CID']].groupby('CASRN').apply(len).describe()

count    715.000000
mean       2.650350
std        2.040414
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max       13.000000
dtype: float64

Those numbers are the summary statistics for the number of CIDs associated with each CASRN. So, there are, **on average, more CIDs returned per CASRN from the API than from the IXS.** This probably means that the API search is less discriminating.

The real question, though, is about the quality of these matches.


## Can we use these data to establish firm CASRN-CID correspondences?

Not sure yet. Note: If we could, then the validity of the original list of CASRNs (which came from California EPA) would also limit the validity of the correspondences. I'm reasonably confident in Cal/EPA's list, though.

Well, here's a start, anyway: We can merge the two sets of results. If we also bring back the names of the chemicals that California EPA was identifying with each CASRN, then we can see for ourselves whether each correspondence makes sense.

In [11]:
name_index = p65.set_index('CASRN')

In [13]:
merged = pd.merge(p65_cids_ixs, p65_cids_api, on='CASRN')
merged['p65name'] = merged['CASRN'].apply(lambda x: name_index.ix[x, 'ChemName'])
print(len(merged))
merged.head()

3839


,CASRN,CID_ixs,CID,IUPACName,InChIKey,MolecularFormula,p65name
0,100-00-5,7474,7474,1-chloro-4-nitrobenzene,CZGCEKJOLUNIFY-UHFFFAOYSA-N,C6H4ClNO2,1-Chloro-4-nitrobenzene
1,100-00-5,7474,101111,"(2S)-2-[(3,5-dinitrobenzoyl)amino]-4-methylpen...",DIOBIOPCRMWGAT-NSHDSACASA-N,C13H15N3O7,1-Chloro-4-nitrobenzene
2,100-25-4,7492,7492,"1,4-dinitrobenzene",FYFDQJRXFWGIBS-UHFFFAOYSA-N,C6H4N2O4,p-Dinitrobenzene
3,100-40-3,7499,7499,4-ethenylcyclohexene,BBDKZWKEPDTENS-UHFFFAOYSA-N,C8H12,4-Vinylcyclohexene
4,100-41-4,7500,7500,ethylbenzene,YNQLUTRBYVCPMQ-UHFFFAOYSA-N,C8H10,Ethylbenzene


### Look for agreement between PUG- and IXS-returned CIDS

We can simply eliminate everything where PUG API and the IXS gave us *different* CIDs. This doesn't mean that those other CIDs are wrong, or that these are right. But this gives me more confidence and seems like a common-sense approach. 

In [14]:
merged_filtered = merged[merged.CID_ixs == merged.CID]
print(len(merged_filtered))
merged_filtered.head()

973


,CASRN,CID_ixs,CID,IUPACName,InChIKey,MolecularFormula,p65name
0,100-00-5,7474,7474,1-chloro-4-nitrobenzene,CZGCEKJOLUNIFY-UHFFFAOYSA-N,C6H4ClNO2,1-Chloro-4-nitrobenzene
2,100-25-4,7492,7492,"1,4-dinitrobenzene",FYFDQJRXFWGIBS-UHFFFAOYSA-N,C6H4N2O4,p-Dinitrobenzene
3,100-40-3,7499,7499,4-ethenylcyclohexene,BBDKZWKEPDTENS-UHFFFAOYSA-N,C8H12,4-Vinylcyclohexene
4,100-41-4,7500,7500,ethylbenzene,YNQLUTRBYVCPMQ-UHFFFAOYSA-N,C8H10,Ethylbenzene
5,100-44-7,7503,7503,chloromethylbenzene,KCXMKQUNVWSEMD-UHFFFAOYSA-N,C7H7Cl,Benzyl chloride


How many of the CASRNs from the original Prop 65 list are here? There were 715 originally. And what about the degree of one-to-many-ness?...

In [18]:
merged_filtered[['CASRN', 'CID']].groupby('CASRN').apply(len).describe()

count    694.000000
mean       1.402017
std        1.286424
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max       13.000000
dtype: float64

Looks similar to the IXS dataset by itself.

We can use this set as the basis for further attempts at refining the ID alignments...

In [19]:
merged_filtered.to_csv('../results/p65_CASRN_CIDmatches.csv')

## To do

From this point, there isn't a whole lot else that I could do with data alone, that would give me high confidence in the alignments. 
* I could use CIR to convert `p65name` to an InChIKey, and then check for equivalence with the InChIKey from PubChem. Supposedly CIR uses [OPSIN](http://opsin.ch.cam.ac.uk/) to parse chemical names.

To get further beyond this, into the realm of accuracy, what I think is needed is a graphical tool for going through the alignments one by one and checking them. It would be tedious but definitely more certain.